In [4]:
import os
import json
import torch
from transformers import BartForConditionalGeneration, PreTrainedTokenizerFast
import evaluate

# 모델 / 토크나이저 로드 경로 지정
MODEL_DIR = "summary_model_final"

def test_model(input_text, model_dir=MODEL_DIR, max_source_length=1024, max_target_length=128):
    """
    학습 완료된 KoBART 모델로 요약을 수행하는 함수
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 1) 토크나이저 / 모델 불러오기
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_dir)
    model = BartForConditionalGeneration.from_pretrained(model_dir)
    model.to(device)
    model.eval()

    # 2) 입력 텍스트 토큰화
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=max_source_length,
        truncation=True
    ).to(device)

    # 3) 요약 생성 (샘플링 or 빔서치)
    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            num_beams=4,
            max_length=max_target_length,
            early_stopping=True
            # 또는 샘플링 모드
            # do_sample=True, top_k=50, top_p=0.95
        )

    # 4) 디코딩
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary_text


if __name__ == "__main__":
    # 평가용 메트릭 (ROUGE)
    rouge = evaluate.load("rouge")

    # 테스트용 입력(들)을 정의
    test_texts = [
        """ 
        오늘 날씨가 너무 좋아서 점심 먹고 산책을 할까 고민 중이야. 아침에는 약간 흐렸는데, 오후 들어서 해가 나더라.
        """,
        """
        데이터 사이언스는 다양한 분야에서 활용된다. 빅데이터 시대에는 방대한 정보를 잘 가공해 인사이트를 뽑아내는 것이 중요하다.
        """
    ]

    # 실제 테스트 실행
    predictions = []
    for text in test_texts:
        summary = test_model(text)
        predictions.append(summary)
        print("===== 원본 텍스트 =====")
        print(text.strip())
        print("===== 요약 결과 =====")
        print(summary)
        print("")

    # (선택) 참조 요약문이 있다면 ROUGE 평가
    # references = ["해당 문장에 대한 참조 요약1", "참조 요약2"]
    # results = rouge.compute(predictions=predictions, references=references)
    # print("ROUGE Results:", results)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


===== 원본 텍스트 =====
오늘 날씨가 너무 좋아서 점심 먹고 산책을 할까 고민 중이야. 아침에는 약간 흐렸는데, 오후 들어서 해가 나더라.
===== 요약 결과 =====
산책을 할까 고민 중이며 아침에는 흐렸다가 오후 들어서 해가 나곤 한다.

===== 원본 텍스트 =====
데이터 사이언스는 다양한 분야에서 활용된다. 빅데이터 시대에는 방대한 정보를 잘 가공해 인사이트를 뽑아내는 것이 중요하다.
===== 요약 결과 =====
빅데이터 시대에에는 방대한 정보를 가공해 인사이트를 뽑아내는 데이터 사이언스가 다양한 분야에서 활용된다.



In [5]:
import os
import json
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import evaluate

# 1. 모델 / 토크나이저 로드 경로 지정
MODEL_DIR = "./summary_model_final"  # 학습이 끝난 모델이 저장된 디렉토리

def test_model(input_text, model_dir=MODEL_DIR, max_source_length=1024, max_target_length=128):
    """
    학습 완료된 Pegasus 모델로 요약을 수행하는 함수
    """
    # GPU 사용 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 1) 토크나이저 / 모델 불러오기
    tokenizer = PegasusTokenizer.from_pretrained(model_dir)
    model = PegasusForConditionalGeneration.from_pretrained(model_dir)
    model.to(device)
    model.eval()

    # 2) 입력 텍스트 토큰화
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        max_length=max_source_length,
        truncation=True
    ).to(device)

    # 3) 요약 생성
    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            do_sample=True,      # 샘플링 모드
            top_k=50, 
            top_p=0.95,
            max_length=256
        )

    # 4) 디코딩
    summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary_text


if __name__ == "__main__":
    # 2. 평가용 메트릭 (ROUGE) 로드
    rouge = evaluate.load("rouge")

    # 3. 테스트용 입력(들)을 정의
    test_texts = [
        """ 
        Alice: 안녕, 오늘 날씨가 정말 좋지 않아?
        Bob: 그러게, 어제는 비가 오더니 오늘은 맑아서 기분이 좋아!
        Alice: 나도! 이렇게 날씨 좋은 날엔 산책이 최고인 것 같아.
        Bob: 맞아, 이따 점심 먹고 같이 산책이나 할까?
        Alice: 그러자. 어디 갈지 생각해둔 데 있어?
        Bob: 그냥 집 근처 공원에 가면 좋을 것 같아. 가는 길에 카페도 있으니까.
        """,
        """
        오늘은 주요 국가들의 경제 상황과 전망에 대해 살펴보겠습니다. 
        지난 분기에 비해 수출은 소폭 감소했지만, 내수시장이 여전히 활발하게 유지되는 모습입니다. 
        전문가들은 하반기에도 비슷한 추세가 이어질 것으로 예상하고 있습니다.
        """
    ]

    # 4. 실제 테스트 실행
    predictions = []
    for text in test_texts:
        summary = test_model(text)  # 추론
        predictions.append(summary)
        print("===== 원본 텍스트 =====")
        print(text.strip())
        print("===== 요약 결과 =====")
        print(summary)
        print("")

    # 5. (선택) 만약 참조 요약문이 있다면, ROUGE 평가도 가능
    #    예: references = ["참조 요약1", "참조 요약2"]
    #    results = rouge.compute(predictions=predictions, references=references)
    #    print("ROUGE Results:", results)


===== 원본 텍스트 =====
Alice: 안녕, 오늘 날씨가 정말 좋지 않아?
        Bob: 그러게, 어제는 비가 오더니 오늘은 맑아서 기분이 좋아!
        Alice: 나도! 이렇게 날씨 좋은 날엔 산책이 최고인 것 같아.
        Bob: 맞아, 이따 점심 먹고 같이 산책이나 할까?
        Alice: 그러자. 어디 갈지 생각해둔 데 있어?
        Bob: 그냥 집 근처 공원에 가면 좋을 것 같아. 가는 길에 카페도 있으니까.
===== 요약 결과 =====
Alice:                .

===== 원본 텍스트 =====
오늘은 주요 국가들의 경제 상황과 전망에 대해 살펴보겠습니다. 
        지난 분기에 비해 수출은 소폭 감소했지만, 내수시장이 여전히 활발하게 유지되는 모습입니다. 
        전문가들은 하반기에도 비슷한 추세가 이어질 것으로 예상하고 있습니다.
===== 요약 결과 =====
.

